# Short-Term Memory in LangGraph with `InMemorySaver` and `SqliteSaver`

This notebook demonstrates how **short-term memory** can be implemented
in **LangGraph** using checkpointing.

The focus is on:
- Preserving conversational state across graph executions
- Summarizing long conversations to control context length
- Comparing in-memory vs SQLite-backed persistence


## Memory in LangGraph

LangGraph supports **checkpointing**, which allows graph state to be:
- Stored between node executions
- Recovered after interruptions
- Scoped to individual conversation threads

Two checkpointing strategies are demonstrated:
- `InMemorySaver` (ephemeral, session-bound)
- `SqliteSaver` (persistent, disk-backed)


In [ ]:
import getpass
import os
from langgraph.graph import START, END, StateGraph, MessagesState
from langchain_core.messages import HumanMessage, BaseMessage, AIMessage, RemoveMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver
import sqlite3
from langgraph.checkpoint.sqlite import  SqliteSaver

In [ ]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

## Defining the Graph State

The state extends `MessagesState`, which already includes
a reducer for accumulating messages.

An additional `summary` field is added to maintain
a compact representation of the conversation.


In [ ]:
class State(MessagesState):
    summary: str

## Chat Model Initialization

A deterministic chat model is used to ensure
stable summaries and responses.


In [ ]:
chat = ChatOpenAI(
    model="gpt-5-nano", 
    temperature=0, 
    model_kwargs= {"text":{"verbosity": 'low'}},
    ) 


## Node: Ask Question

This node prompts the user for a new question
and appends it to the message history.


In [ ]:
def ask_question(state: State) -> State:
    
    print(f"\n-------> ENTERING ask_question:")
    
    question = "What is your question?"
    print(question)
    
    return State(messages = [AIMessage(question), HumanMessage(input())])

## Node: Chatbot

This node:
- Reads the full message history
- Injects the running summary as a system message
- Generates a context-aware response


In [ ]:
def chatbot(state: State) -> State:
    
    print(f"\n-------> ENTERING chatbot:")
    for i in state["messages"]:
        i.pretty_print()
        
    system_message = f'''
    Here's a quick summary of what's been discussed so far:
    {state.get("summary", "")}
    
    Keep this in mind as you answer the next question.
    '''
    
    response = chat.invoke([SystemMessage(system_message)] + state["messages"])
    response.pretty_print()
    
    return State(messages = [response])

## Node: Summarize Messages

To prevent unbounded context growth, this node:
- Summarizes the recent conversation
- Updates the running summary
- Removes detailed message history using `RemoveMessage`


In [ ]:
def summarize_messages(state: State) -> State:
    print(f"\n-------> ENTERING summarize_messages:")
    
    new_conversation = ""
    for i in state["messages"]:
        new_conversation += f"{i.type}: {i.content}\n\n"
        
    summary_instructions = f'''
Update the ongoing summary by incorporating the new lines of conversation below.  
Build upon the previous summary rather than repeating it so that the result  
reflects the most recent context and developments.


Previous Summary:
{state.get("summary", "")}

New Conversation:
{new_conversation}
'''
    
    print(summary_instructions)
    
    summary = chat.invoke([HumanMessage(summary_instructions)])
    
    remove_messages = [RemoveMessage(id = i.id) for i in state["messages"][:]]
    
    return State(messages = remove_messages, summary = summary.content)

## Defining the Graph

The graph follows a simple conversational flow:
1. Ask a question
2. Generate a response
3. Summarize and prune message history


In [ ]:
graph = StateGraph(State)

In [ ]:
graph.add_node("ask_question", ask_question)
graph.add_node("chatbot", chatbot)
graph.add_node("summarize_messages", summarize_messages)

graph.add_edge(START, "ask_question")
graph.add_edge("ask_question", "chatbot")
graph.add_edge("chatbot", "summarize_messages")
graph.add_edge("summarize_messages", END)

## Checkpointing and Persistence

Two checkpointing strategies are supported:

- `InMemorySaver`  
  - State persists only for the current kernel session

- `SqliteSaver`  
  - State is persisted to disk
  - Enables recovery across executions


In [ ]:
db_path =  "./LangGraph_DB/langgraph.db"
con = sqlite3.connect(database=db_path, check_same_thread=False)

In [ ]:
# checkpointer =  InMemorySaver()  #for kernal sessions memory persistence
checkpointer =  SqliteSaver(con)

graph_compiled = graph.compile(checkpointer)

## Compiling and Executing the Graph

Checkpointing is enabled during compilation.
Each conversation is scoped using a `thread_id`.


In [ ]:
graph_compiled

In [ ]:
config1 = {"configurable" : {"thread_id": "1"}}

In [ ]:
graph_compiled.invoke(State(),config1)   #Instructs the graphs to create in-memory checkpoint and associate them with the specific thread, enabling thread-level persistence.

## Inspecting Checkpointed State History

LangGraph allows inspection of all intermediate
states stored during execution.


In [ ]:
graph_states = [i for i in graph_compiled.get_state_history(config1)]

In [ ]:
graph_states

In [ ]:
for i in graph_states[::-1]:
    print(f'''
    Messages: {i.values['messages']}
    Summary: {i.values.get('summary', '')}
    Next: {i.next}
    Step: {i.metadata["step"]}
    ''')

## Summary

This notebook demonstrated short-term memory in LangGraph by:

- Accumulating conversation messages using `MessagesState`
- Summarizing and pruning history to manage context size
- Using checkpointing for state persistence
- Comparing in-memory and SQLite-backed storage

This pattern enables scalable, multi-turn conversational
systems with controlled memory growth.
